# $$Scalable\ Kmeans\ Plus\ Plus$$
## $$Final\ Project\ of\ STA663$$
### $$Jiancong\ Zhu\ \&\ William\ Eastman$$

# Part I
# Introduction and Background:  

**Long-popular and ubiquitous in its application, k-means clustering remains one of the most widely used unsupervised learning methods today. Despite its popularity and ease of use, the algorithm, in its most basic form, suffers from several theoretical limitations. Its solutions, though locally optimal, can be very far from global optima even under repeated random initialization. Certain initializations also result in computational issues.** 
  
**When the initial set of centers is suboptimal, the final solution can be very poor or not even converge. Therefore, a proper initialization of k-means is crucial for obtaining a good final solution. Compared with selecting the initial set of centers randomly, the more recently proposed kmeans++ initialization algorithm is superior and can obtain an initial set of centers that is provably close to the optimum solution. However, kmeans++ initialization algorithm also has its disadvantage, with its major downside being its inherent sequential nature, which limits its applicability to massive data. A naive implementation of k-means++ initialization will make k passes over the data. When the data is massive, then k may be also very large and the k-means++ initialization would be very slow as a result.**
  
**The paper introduces an parallel version of k-means++, which is called scalable kmeans++ initialization algorithm or k-means|| initialization algorithm. K-means|| can somewhat solve the downside of k-means++ described above. K-means++ initialization algoritm produces one center each round, while k-means|| may produce several centers each iteration.** 
  
**In this project, we will perform k-means algorithm on both synthetic datasets and real world data with initial set of centers simulated by random selecting initialization algorithm, k-means++ initialization algorithm and k=means|| initialization algorithm respectively.** 

**We will implement these three initialization algorithm and compare the results of them.**

### K-means Clustering:

**Let X={$x_1,...,x_n$} be a set of points in the d-dimensional Euclidean spance and let k be a positive integer specifying the number of clusters.**

**Let $||x_i-x_j||$ donate the Euclidean distance between $x_i$ and $x_j$. For a point $x$ and a subset $Y \in X$ of points, the distance is defined as $d(x,Y)=min_{y \in Y}||x-y||$.**

**For a subset $Y \in X$ of points, the centroid is given by $$centroid(Y)=\frac{1}{|Y|}\Sigma _{y \in Y} y$$**

**Let $C$={c_1,...,c_k} be a set of points and let $Y \in X$. We define the cost of Y with respect to $C$ as $$\phi _Y(C)=d^2(y,C)=\Sigma_ {y \in Y} min_{i=1,...,k} ||y-c_i||^2$$**

**The goal of k-means clustering is to choose a set of $C$ of k centers to minimize $\phi _X(C).$**

$$\\\\$$

**As discussed above, the initial set of centers has big effects on the k-means result. Generally, the smaller the $\phi _X(C)$, the better k-means result we get.**

**When the dataset and initial set of centers are given, apply Lloyd iterations to the them until the max iterations or convergence.**

**Therefore, the initial set of centers we want to get is that, after Lloyd iterations, the final $\phi _X(C)$ will be as same as possible.**

## Import Modules:

In [235]:
import numpy as np
import pandas as pd
from numba import jit

# Part II
# Data Preparation:

### 1. Synthetic Dataset:

**We construct the gaussmix function to simulate the synthetic dataset--GaussMixture dataset.**



**Process of simulating the GaussMixture dataset:**

**1. Simulate k centers with mean as the origin and sphere variance as r.**

**2. For each center of the k centers, add n points to it with unit sphere variance.**


**Therefore, k stands for the number of clusters, d stands for the dimension, n stands for the points added to each center and r is the sphere variance of the simulated k centers. The total number of points in the simulated dataset is $k*(n+1)$.**

In [236]:
#GaussMitxture
#simulate data
#k is the number of simulated centroids
#d is the dimension
#n is the number of points simulated around each centroid
#r is the sphere variance for simulated centroids
def gaussmix(k,d,n,r):
    #expected value of each centroid
    mean=np.zeros(d)
    #coviance matrix of simulated centroids
    covk=np.diag([r]*d)
    #simulate k centroids
    centers=np.random.multivariate_normal(mean,covk,k)
    #coviance matrix of points simulated around each centroid
    covu=np.diag([1]*d)
    #simulate n points around each centroid
    data=np.random.multivariate_normal(centers[0],covu,n)
    labels=list(range(1,k+1))+[1]*n
    for i in range(1,k):
        data=np.r_[data,np.random.multivariate_normal(centers[i],covu,n)]
        labels.extend([i+1]*n)
    return centers,np.r_[centers,data],np.array(labels)

**To make the results same for each time so that the paper can be reproduced, we set a seed.**

In [237]:
np.random.seed(1)

**GaussMixture dataset with k=20, d=15, n=499 and r=1.**

In [238]:
%%time
centers1,gmdata1,gmlabel1=gaussmix(20,15,499,1)

CPU times: user 412 ms, sys: 8 ms, total: 420 ms
Wall time: 63.6 ms


**GaussMixture dataset with k=20, d=15, n=499 and r=10.**

In [239]:
%%time
centers10,gmdata10,gmlabel10=gaussmix(20,15,499,10)

CPU times: user 148 ms, sys: 0 ns, total: 148 ms
Wall time: 19.1 ms


**GaussMixture dataset with k=20, d=15, n=499 and r=100.**

In [240]:
%%time
centers100,gmdata100,gmlabel100=gaussmix(20,15,499,100)

CPU times: user 288 ms, sys: 16 ms, total: 304 ms
Wall time: 39.4 ms


### Real world dataset:

**The real world dataset we use is the spambase dataset. It has 4601 points and the dimension of each point is 58.**

In [241]:
#spambase
spam=pd.read_csv('spambase.data', header=None)
spam=np.array(spam)
spam.shape

(4601, 58)

# Part II 
# Implementation of Algorithms:

### 1. Implement the Lloyd Iteration Algorithm:

**Lloyd Iteration Algorithm:**

**1. Start with a set of randomly chosen initial centers**

**2. Assign each input point to its nearest center**

**3. Recoumpute the centers given the point assignment**

**4. Repeat steps 2 and 3 until the solution does not change**

In [242]:
#given all the points and the current centroids, then cluster the points and label them
def relabel(data,centroid):
    n=len(data)
    k=len(centroid)
    dist=np.zeros((n,k))
    for i in range(n):
        for j in range(k):
            dist[i,j]=np.sum((data[i]-centroid[j])**2)
    return np.argmin(dist,axis=1)+1

In [243]:
#given all the points and their labels, then get the new centroids
def recentroid(data,label):
    k=len(np.unique(label))
    p=data.shape[1]
    centroid=np.zeros((k,p))
    for i in range(k):
        centroid[i]=np.mean(data[label==i+1],axis=0)
    return centroid

In [244]:
#given all the points and initial centroid, then do Lloyd iterations until convergence or max iterations
#then return the centroids, labels and iteration times
def Lloyd(data,centroid,imax=1000):
    i=1
    k=len(centroid)
    n=len(data)
    centroid1=centroid
    label=relabel(data,centroid1)
    centroid2=recentroid(data,label)
    while ((np.array_equal(centroid1,centroid2)==False) and i<imax):
        i=i+1
        label=relabel(data,centroid2)
        centroid1=centroid2.copy()
        centroid2=recentroid(data,label)
    return centroid2, label,i

### 2. Implement K-means Algorithm with Random Initialization:

**Randomly Initialized Algorithm:**

**This algorithm will select k points as the initial set of centers randomly from the points in the data.**

In [245]:
#select k centroids randomly
def kmeans(data,k):
    n=len(data)
    select=np.random.choice(n,k,replace=False)
    return data[select]

### 3. Implement the Algorithm with K-means++ Initialization:

**The random  initialization algorithm selects the points with equal probability, so the initial set of centers are selected completely randomly.**

**However, the k-means++ initialization algorithm selects points with different probability. This algorithm selects exactly one point into the initial set of centers each iteration. The probability of $x \in X$ being selected is $\frac{d^{2}(x,C)}{\phi_{X}(C)}$, where $C$ is the current set of centers.**

**Therefore, the probability that $x$ is selected is proportional to its square distance to the current set of centers. The points have been in the current set of centers will not be selected because its square distance to the current set of centers is 0.**

## K-means++ Initialization Algorithm Steps:

**1: Sample a point uniformly at random from X and let it be $C$**

**2: while $|C|<k$ do**

**3: Sample $x \in C$ with probability $\frac{d^{2}(x,C)}{\phi_{X}(C)}$**

**4: Add $x$ into $C$**

**5: end while**

**The $C$ we get is the initial set of centers produced by kmeans++ initialization algorithm.**

In [246]:
#calculate the distance square between the point to given centroids
def distance(x,centroid):
    return min(np.sum((x-centroid)**2,axis=1))

In [247]:
#calculate the cost of the given points with respect to given centroids
def cost(data, centroid):
    cos=0
    for i in range(len(data)):
        cos=cos+distance(data[i],centroid)
    return cos

In [248]:
#calulate the probablity that each point is selected into the initial centroids for both kmeans++
#and kmeans||
def distribution(data,centroid,l=1):
    n=len(data)
    distri=np.zeros(n)
    tcost=cost(data,centroid)
    for i in range(n):
        p=l*distance(data[i],centroid)/tcost
        if p<=1:
            distri[i]=p
        else:
            distri[i]=1
    return distri

In [249]:
#kmeans++ initialization algorithm using for loop
def kmeanspp(data,k):
    n=len(data)
    samples=np.array(list(range(n)))
    select=np.random.choice(samples,1)
    centroid=data[select]
    while len(centroid)<k:
        select=np.random.choice(samples,1,p=distribution(data,centroid))
        centroid=np.r_[centroid, data[select]]
    return centroid

### Implement the K-means|| Algorithm:

**k-means|| algorithm steps:**

**The k-means|| algorithm is a parallel version of the k-means++ initialization algorithm. As discussed above, when the dataset is massive and k is large, k-means++ will not be as proper as for normal size dataset. It selects only one point each iteration and for each iteration, all the data will be passed. The running time of k-means++ for massive dataset will be very long.**

**For the k-means|| algorithm, there is a controling parameter l. For k-means++, the sum of probabilities of each point to be selected is 1 and for each iteration, exactly one point will be selected. However, for kmeans++, the probability of each point to be selected is $l\frac{d^{2}(x,C)}{\phi_{X}(C)}$ and that whether a point will be selected is independent on other points. Therefore, theoretically, for each iteration, it may select all the points that not in the current set of centers or select no point. The expected value of number of selected points in each iteration is l.**

**To be convenient, I use a paramater r to control the iteration times.**

**After r iterations, more than k points will be selected and we call the set of these points C.**

**For each point in C, let its weight as the number of points closer to it in X, then C is a weighted set of centers.**

**Recluster C into k clusters and the set of the k centers of these k clusters will be the initial set of centers produced by kmeas|| algorithm.**


## K-means|| Initialization Algorithm Steps:

**1: Sample a point uniformly at random from X and let it be $C$**

**2: For r times do**

**3: Sample each point $x \in X$ independently with probability $l\frac{d^{2}(x,C)}{\phi_{X}(C)}$**

**4: Add the sampled points into $C$**

**5: end for**

**6: For $x \in C$, set w_x to be the number of points in X closer to x than any other point in $C$**

**7: Recluster the weighted points in $C$ into k clusters**

**For step 7, the reclustering method we choose in this project is weighted kmeans++ initialization algorithm.**

**The reclustered set of centers is the initial set of centers produced by kmeans|| initialization algorithm.**

In [250]:
#calculate the distance square between the weighted point to given centroids
def weighteddistance(x,centroid,weight):
    return min(weight*np.sum((x-centroid)**2,axis=1))

In [251]:
#calculate the cost of the given weighted points with respect to given centroids 
def weightedcost(data,centroid,weight):
    cos=0
    for i in range(len(data)):
        cos=cos+weighteddistance(data[i],centroid,weight[i])
    return cos

In [252]:
#calulate the probablity that each weighted point is selected into the initial centroids for kmeans++
def weighteddistribution(data,centroid,weight):
    n=len(data)
    distri=np.zeros(n)
    wcost=weightedcost(data,centroid,weight)
    for i in range(n):
        distri[i]=weighteddistance(data[i],centroid,weight[i])/wcost
    return distri

In [253]:
#kmeans++ initialization algorithm for weighted points
def weightedkmeanspp(data,k,weight):
    n=len(data)
    samples=np.array(list(range(n)))
    select=np.random.choice(samples,1)
    centroid=data[select]
    while len(centroid)<k:
        select=np.random.choice(samples,1,p=weighteddistribution(data,centroid,weight))
        centroid=np.r_[centroid,data[select]]
    return centroid        

In [254]:
#recluster the weighted points in the given centroids
def recluster(data,k,centroid):
    c=len(centroid)
    weight=np.zeros(c)
    label=relabel(data,centroid)
    for i in range(c):
        weight[i]=np.sum(label==(i+1))
    return weightedkmeanspp(centroid,k,weight)    

In [255]:
#kmeans|| initialization algorithm
def kmeanspar(data,k,l,r):
    n=len(data)
    samples=np.array(list(range(n)))
    select=np.random.choice(samples,1)
    centroid=data[select]
    for i in range(r):
        select=samples[np.random.binomial(1,p=distribution(data,centroid,l))==1]
        centroid=np.r_[centroid, data[select]]
        if len(centroid)==n:
            return recluster(data,k,centroid)
    return recluster(data,k,centroid)

# Part III
# Testing:

**In this part, we test the functions in the algorithms above using the three GaussMixture datasets and the spambase data set.**

#### 1. test relabel function:

In [256]:
%%time
relabel(gmdata1,centers1)

CPU times: user 1.56 s, sys: 0 ns, total: 1.56 s
Wall time: 1.56 s


array([ 1,  2,  3, ..., 20, 20, 20])

In [257]:
%%time
relabel(gmdata10,centers10)

CPU times: user 1.59 s, sys: 4 ms, total: 1.59 s
Wall time: 1.6 s


array([ 1,  2,  3, ..., 20, 20, 20])

In [258]:
%%time
relabel(gmdata100,centers100)

CPU times: user 1.68 s, sys: 0 ns, total: 1.68 s
Wall time: 1.68 s


array([ 1,  2,  3, ..., 20, 20, 20])

#### 2. test recentroid function:

In [259]:
%%time
c1=recentroid(gmdata1, gmlabel1)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 2.86 ms


In [260]:
%%time
c5=recentroid(gmdata10, gmlabel10)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 2.41 ms


In [261]:
%%time
c10=recentroid(gmdata100, gmlabel100)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 2.16 ms


#### 3. test Lloyd iteration function:

In [262]:
%%time
L1=Lloyd(gmdata1,centers1)

CPU times: user 11.1 s, sys: 0 ns, total: 11.1 s
Wall time: 11.2 s


In [263]:
%%time
L5=Lloyd(gmdata10,centers10)

CPU times: user 3.04 s, sys: 4 ms, total: 3.05 s
Wall time: 3.05 s


In [264]:
%%time
L10=Lloyd(gmdata100,centers100)

CPU times: user 3.14 s, sys: 4 ms, total: 3.15 s
Wall time: 3.15 s


#### 4. test the random selecting initialization algorithm:

In [265]:
%%time
rs1=kmeans(gmdata1,20)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 365 µs


In [266]:
%%time
rs5=kmeans(gmdata10,20)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 361 µs


In [267]:
%%time
rs10=kmeans(gmdata100,20)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 396 µs


In [268]:
%%time
rsp=kmeans(spam,20)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 345 µs


#### 5. test the cost function:

In [269]:
%%time
cost(gmdata1,centers1)

CPU times: user 164 ms, sys: 0 ns, total: 164 ms
Wall time: 170 ms


144818.64575706958

In [270]:
%%time
cost(gmdata10,centers10)

CPU times: user 160 ms, sys: 4 ms, total: 164 ms
Wall time: 167 ms


150030.54718866557

In [271]:
%%time
cost(gmdata100,centers100)

CPU times: user 160 ms, sys: 0 ns, total: 160 ms
Wall time: 164 ms


150005.6030282646

#### 6. test the distribution function:

**The sum of the returned vector by this function should be 1 (or very very close to 1 in case of calculation accuray).**

In [272]:
%%time
np.sum(distribution(gmdata1,centers1))

CPU times: user 324 ms, sys: 4 ms, total: 328 ms
Wall time: 332 ms


1.0000000000000031

In [273]:
%%time
np.sum(distribution(gmdata10,centers10))

CPU times: user 336 ms, sys: 4 ms, total: 340 ms
Wall time: 349 ms


0.99999999999999989

In [274]:
%%time
np.sum(distribution(gmdata100,centers100))

CPU times: user 344 ms, sys: 0 ns, total: 344 ms
Wall time: 344 ms


0.99999999999999956

#### 7. test the kmeans++ initialization algorithm function:

In [275]:
%%time
kpp1=kmeanspp(gmdata1,20)

CPU times: user 5.57 s, sys: 8 ms, total: 5.58 s
Wall time: 5.62 s


In [276]:
%%time
kpp5=kmeanspp(gmdata10,20)

CPU times: user 5.51 s, sys: 20 ms, total: 5.53 s
Wall time: 5.58 s


In [277]:
%%time
kpp10=kmeanspp(gmdata100,20)

CPU times: user 5.56 s, sys: 8 ms, total: 5.57 s
Wall time: 5.58 s


In [278]:
%%time
kppsp=kmeanspp(spam,20)

CPU times: user 2.96 s, sys: 0 ns, total: 2.96 s
Wall time: 2.96 s


#### 8. test the weightedcost function:

**The results of weightedcost function here are same as cost function above because the weights are same for every point**

In [279]:
%%time
weightedcost(gmdata1,centers1,np.array([1]*10000))

CPU times: user 208 ms, sys: 0 ns, total: 208 ms
Wall time: 207 ms


144818.64575706958

In [280]:
%%time
weightedcost(gmdata10,centers10,np.array([1]*10000))

CPU times: user 192 ms, sys: 0 ns, total: 192 ms
Wall time: 189 ms


150030.54718866557

In [281]:
%%time
weightedcost(gmdata100,centers100,np.array([1]*10000))

CPU times: user 216 ms, sys: 0 ns, total: 216 ms
Wall time: 215 ms


150005.6030282646

#### 9. test the weighteddistribution function:

**The sum of the returned vector by this function should be 1 (or very very close to 1 in case of calculation accuray).**

In [282]:
%%time
np.sum(weighteddistribution(gmdata1,centers1,np.array([1]*10000)))

CPU times: user 392 ms, sys: 0 ns, total: 392 ms
Wall time: 392 ms


1.0000000000000031

In [283]:
%%time
np.sum(weighteddistribution(gmdata10,centers10,np.array([1]*10000)))

CPU times: user 364 ms, sys: 4 ms, total: 368 ms
Wall time: 370 ms


0.99999999999999989

In [284]:
%%time
np.sum(weighteddistribution(gmdata100,centers100,np.array([1]*10000)))

CPU times: user 408 ms, sys: 0 ns, total: 408 ms
Wall time: 409 ms


0.99999999999999956

#### 10. test the kmeans++ initialization algorithm function for weighted points:

In [285]:
%%time
wkpp1=weightedkmeanspp(gmdata1,20,np.array([1]*10000))

CPU times: user 6.67 s, sys: 4 ms, total: 6.67 s
Wall time: 6.67 s


In [286]:
%%time
wkpp5=weightedkmeanspp(gmdata10,20,np.array([1]*10000))

CPU times: user 6.66 s, sys: 4 ms, total: 6.66 s
Wall time: 6.67 s


In [287]:
%%time
wkpp10=weightedkmeanspp(gmdata100,20,np.array([1]*10000))

CPU times: user 6.33 s, sys: 0 ns, total: 6.33 s
Wall time: 6.33 s


In [288]:
%%time
wkppsp=weightedkmeanspp(spam,20,np.array([1]*4601))

CPU times: user 3.42 s, sys: 4 ms, total: 3.43 s
Wall time: 3.43 s


#### 11. test the recluster function:

In [289]:
%%time
re1=recluster(gmdata1,5,centers1)

CPU times: user 1.5 s, sys: 0 ns, total: 1.5 s
Wall time: 1.5 s


In [290]:
%%time
re5=recluster(gmdata10,5,centers10)

CPU times: user 1.51 s, sys: 0 ns, total: 1.51 s
Wall time: 1.51 s


In [291]:
%%time
re10=recluster(gmdata100,5,centers100)

CPU times: user 1.45 s, sys: 0 ns, total: 1.45 s
Wall time: 1.45 s


#### 12.  test the kmeans|| initialization algorithm function:

In [292]:
%%time
sckpp1=kmeanspar(gmdata1,20,6,6)

CPU times: user 4.24 s, sys: 12 ms, total: 4.25 s
Wall time: 4.27 s


In [293]:
%%time
sckpp5=kmeanspar(gmdata10,20,6,6)

CPU times: user 5.15 s, sys: 12 ms, total: 5.16 s
Wall time: 5.17 s


In [294]:
%%time
sckpp10=kmeanspar(gmdata100,20,6,6)

CPU times: user 5.37 s, sys: 0 ns, total: 5.37 s
Wall time: 5.37 s


In [295]:
%%time
sckppsp=kmeanspar(spam,20,6,6)

CPU times: user 3.28 s, sys: 0 ns, total: 3.28 s
Wall time: 3.28 s


**The functions are appliable for all the four datasets and there is no error or warning.**

**Therefore, the codes are appropriate in this project.**

# Part IV
# Performance Optimization:

## 1. Vectorization Using Broadcasting:

**We use broadcasting to speed up all the functions in the above algorithm and carry out the updated functions with gmdata1. For each updated function, I carry out its corresponding original function above on gmdata1 and compare running times.**

### 1.1 Implement the Lloyd Iteration Algorithm Using Broadcasting:

In [296]:
#using broadcasting instead of forloof
def vrelabel(data,centroid):
    dist=np.sum((data[:,None,:]-centroid)**2,axis=2)
    return np.argmin(dist,axis=1)+1

In [297]:
%%time
vrelabel(gmdata1,centers1)

CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 19.4 ms


array([ 1,  2,  3, ..., 20, 20, 20])

In [298]:
%%time
relabel(gmdata1,centers1)

CPU times: user 1.71 s, sys: 0 ns, total: 1.71 s
Wall time: 1.71 s


array([ 1,  2,  3, ..., 20, 20, 20])

In [299]:
#using broadcasting vrelabel
def vLloyd(data,centroid,imax=1000):
    i=1
    k=len(centroid)
    n=len(data)
    centroid1=centroid
    label=vrelabel(data,centroid1)
    centroid2=recentroid(data,label)
    while ((np.array_equal(centroid1,centroid2)==False) and i<imax):
        i=i+1
        label=vrelabel(data,centroid2)
        centroid1=centroid2.copy()
        centroid2=recentroid(data,label)
    return centroid2, label,i

In [300]:
%%time
vL1=vLloyd(gmdata1,centers1)

CPU times: user 144 ms, sys: 0 ns, total: 144 ms
Wall time: 145 ms


In [301]:
%%time
L1=Lloyd(gmdata1,centers1)

CPU times: user 11 s, sys: 0 ns, total: 11 s
Wall time: 11 s


### 1.2 Implement the K-means++ Initialization Algorithm Using Broadcasting:

In [302]:
#calculate the distance square between each point to given centroids
def vdistance(data,centroid):
    dist=np.sum((data[:,None,:]-centroid)**2,axis=2)
    return dist.min(axis=1)

In [303]:
%%time
vdistance(gmdata1,centers1)

CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 18.4 ms


array([  0.        ,   0.        ,   0.        , ...,   8.79177241,
        17.36878434,  20.98049325])

In [304]:
#using broadcasting
def vcost(data, centroid):
    dist=np.sum((data[:,None,:]-centroid)**2,axis=2)
    return np.sum(dist.min(axis=1))

In [305]:
%%time
vcost(gmdata1,centers1)

CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 19.4 ms


144818.64575707004

In [306]:
%%time
cost(gmdata1,centers1)

CPU times: user 164 ms, sys: 4 ms, total: 168 ms
Wall time: 167 ms


144818.64575706958

In [307]:
#using broadcasting
def vdistribution(data,centroid,l=1):
    distri=l*vdistance(data,centroid)/vcost(data,centroid)
    distri[distri>1]=1
    return distri

In [308]:
%%time
np.sum(vdistribution(gmdata1,centers1))

CPU times: user 40 ms, sys: 0 ns, total: 40 ms
Wall time: 37.9 ms


0.99999999999999989

In [309]:
%%time
np.sum(distribution(gmdata1,centers1))

CPU times: user 328 ms, sys: 4 ms, total: 332 ms
Wall time: 337 ms


1.0000000000000031

In [310]:
##kmeans++ initialization algorithm using broadcasting
def vkmeanspp(data,k):
    n=len(data)
    samples=np.array(list(range(n)))
    select=np.random.choice(samples,1)
    centroid=data[select]
    while len(centroid)<k:
        select=np.random.choice(samples,1,p=vdistribution(data,centroid))
        centroid=np.r_[centroid, data[select]]
    return centroid

In [311]:
%%time
vkpp1=vkmeanspp(gmdata1,20)

CPU times: user 348 ms, sys: 0 ns, total: 348 ms
Wall time: 359 ms


In [312]:
%%time
kpp1=kmeanspp(gmdata1,20)

CPU times: user 5.48 s, sys: 12 ms, total: 5.5 s
Wall time: 5.52 s


### 1.3 Implement the K-means|| Initialization Algorithm Using Broadcasting:

In [313]:
#using broadcasting to calculate the distance square between each weighted point to given centroids
def vweighteddistance(data,centroid,weight):
    dist=np.sum((data[:,None,:]-centroid)**2,axis=2)
    wdist=(dist.T*weight).T
    return wdist.min(axis=1)

In [314]:
%%time
vweighteddistance(gmdata1,centers1,np.array([1]*10000))

CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 19.5 ms


array([  0.        ,   0.        ,   0.        , ...,   8.79177241,
        17.36878434,  20.98049325])

In [315]:
def vweightedcost(data,centroid,weight):
    dist=np.sum((data[:,None,:]-centroid)**2,axis=2)
    wdist=(dist.T*weight).T
    return np.sum(wdist.min(axis=1))  

In [316]:
%%time
vweightedcost(gmdata1,centers1,np.array([1]*10000))

CPU times: user 24 ms, sys: 0 ns, total: 24 ms
Wall time: 26.8 ms


144818.64575707004

In [317]:
%%time
weightedcost(gmdata1,centers1,np.array([1]*10000))

CPU times: user 196 ms, sys: 0 ns, total: 196 ms
Wall time: 203 ms


144818.64575706958

In [318]:
def vweighteddistribution(data,centroid,weight):
    distri=vweighteddistance(data,centroid,weight)/vweightedcost(data,centroid,weight)
    return distri

In [319]:
%%time
np.sum(vweighteddistribution(gmdata1,centers1,np.array([1]*10000)))

CPU times: user 40 ms, sys: 0 ns, total: 40 ms
Wall time: 39.7 ms


0.99999999999999989

In [320]:
%%time
np.sum(weighteddistribution(gmdata1,centers1,np.array([1]*10000)))

CPU times: user 420 ms, sys: 4 ms, total: 424 ms
Wall time: 423 ms


1.0000000000000031

In [321]:
#kmeans++ initialization algorithm for weighted points using broadcasting 
def vweightedkmeanspp(data,k,weight):
    n=len(data)
    samples=np.array(list(range(n)))
    select=np.random.choice(samples,1)
    centroid=data[select]
    while len(centroid)<k:
        select=np.random.choice(samples,1,p=vweighteddistribution(data,centroid,weight))
        centroid=np.r_[centroid,data[select]]
    return centroid  

In [322]:
%%time
vwkpp1=vweightedkmeanspp(gmdata1,20,np.array([1]*10000))

CPU times: user 376 ms, sys: 4 ms, total: 380 ms
Wall time: 384 ms


In [323]:
%%time
wkpp1=weightedkmeanspp(gmdata1,20,np.array([1]*10000))

CPU times: user 6.63 s, sys: 0 ns, total: 6.63 s
Wall time: 6.67 s


In [324]:
#recluster the weighted points in the given centroids using broadcasting
def vrecluster(data,k,centroid):
    c=len(centroid)
    weight=np.zeros(c)
    label=vrelabel(data,centroid)
    for i in range(c):
        weight[i]=np.sum(label==i+1)
    return vweightedkmeanspp(centroid,k,weight) 

In [325]:
%%time
vre1=vrecluster(gmdata1,10,centers1)

CPU times: user 24 ms, sys: 0 ns, total: 24 ms
Wall time: 22.2 ms


In [326]:
%%time
re1=recluster(gmdata1,10,centers1)

CPU times: user 1.49 s, sys: 0 ns, total: 1.49 s
Wall time: 1.5 s


In [327]:
#kmeans|| initialization algorithm using broadcasting
def vkmeanspar(data,k,l,r):
    n=len(data)
    samples=np.array(list(range(n)))
    select=np.random.choice(samples,1)
    centroid=data[select]
    for i in range(r):
        select=samples[np.random.binomial(1,p=vdistribution(data,centroid,l))==1]
        centroid=np.r_[centroid, data[select]]
        if len(centroid)==n:
            return vrecluster(data,k,centroid)
    return vrecluster(data,k,centroid)

In [328]:
%%time
vsckpp1=vkmeanspar(gmdata1,20,6,6)

CPU times: user 180 ms, sys: 16 ms, total: 196 ms
Wall time: 197 ms


In [329]:
%%time
sckpp1=kmeanspar(gmdata1,20,6,6)

CPU times: user 4.33 s, sys: 0 ns, total: 4.33 s
Wall time: 4.36 s


**As the results shown above, the broadcasting functions are much quicker than the original functions. The running time is much less after using broadcasting to speed up**

## 2. Using Jit:

In [330]:
@jit
def jit_relabel(data,centroid):
    n=len(data)
    k=len(centroid)
    dist=np.array([0]*(n*k)).reshape(n,k).astype("float")
    for i in range(n):
        for j in range(k):
            dist[i,j]=np.sum((data[i]-centroid[j])**2)
    return np.argmin(dist,axis=1)+1

@jit
def jit_recentroid(data,label):
    k=len(np.unique(label))
    p=data.shape[1]
    centroid=np.array([0]*k*p).reshape(k,p).astype("float")
    for i in range(k):
        centroid[i]=np.mean(data[label==i+1],axis=0)
    return centroid

@jit
def jit_Lloyd(data,centroid,imax=1000):
    i=1
    k=len(centroid)
    n=len(data)
    centroid1=centroid
    label=jit_relabel(data,centroid1)
    centroid2=jit_recentroid(data,label)
    while ((np.array_equal(centroid1,centroid2)==False) and i<imax):
        i=i+1
        label=jit_relabel(data,centroid2)
        centroid1=centroid2.copy()
        centroid2=jit_recentroid(data,label)
    return centroid2, label,i

@jit
def jit_kmeans(data,k):
    n=len(data)
    select=np.random.choice(n,k,replace=False)
    return data[select]

@jit
def jit_distance(x,centroid):
    return min(np.sum((x-centroid)**2,axis=1))

@jit
def jit_cost(data, centroid):
    cos=0
    for i in range(len(data)):
        cos=cos+jit_distance(data[i],centroid)
    return cos

@jit
def jit_distribution(data,centroid,l=1):
    n=len(data)
    distri=np.zeros(n)
    cost=jit_cost(data,centroid)
    for i in range(n):
        p=l*jit_distance(data[i],centroid)/cost
        if p<=1:
            distri[i]=p
        else:
            distri[i]=1
    return distri

@jit
def jit_kmeanspp(data,k):
    n=len(data)
    samples=np.array(list(range(n)))
    select=np.random.choice(samples,1)
    centroid=data[select]
    while len(centroid)<k:
        select=np.random.choice(samples,1,p=jit_distribution(data,centroid))
        centroid=np.r_[centroid, data[select]]
    return centroid

@jit
def jit_weighteddistance(x,centroid,weight):
    return min(weight*np.sum((x-centroid)**2,axis=1))

@jit
def jit_weightedcost(data,centroid,weight):
    cos=0
    for i in range(len(data)):
        cos=cos+jit_weighteddistance(data[i],centroid,weight[i])
    return cos

@jit
def jit_weighteddistribution(data,centroid,weight):
    n=len(data)
    distri=np.zeros(n)
    wcost=jit_weightedcost(data,centroid,weight)
    for i in range(n):
        distri[i]=jit_weighteddistance(data[i],centroid,weight[i])/wcost
    return distri

@jit
def jit_weightedkmeanspp(data,k,weight):
    n=len(data)
    samples=np.array(list(range(n)))
    select=np.random.choice(samples,1)
    centroid=data[select]
    while len(centroid)<k:
        select=np.random.choice(samples,1,p=jit_weighteddistribution(data,centroid,weight))
        centroid=np.r_[centroid,data[select]]
    return centroid

@jit
def jit_recluster(data,k,centroid):
    c=len(centroid)
    weight=np.zeros(c)
    label=jit_relabel(data,centroid)
    for i in range(c):
        weight[i]=np.sum(label==(i+1))
    return jit_weightedkmeanspp(centroid,k,weight)

@jit
def jit_kmeanspar(data,k,l,r):
    n=len(data)
    samples=np.array(list(range(n)))
    select=np.random.choice(samples,1)
    centroid=data[select]
    for i in range(r):
        select=samples[np.random.binomial(1,p=jit_distribution(data,centroid,l))==1]
        centroid=np.r_[centroid, data[select]]
        if len(centroid)==n:
            return jit_recluster(data,k,centroid)
    return jit_recluster(data,k,centroid)

In [331]:
# Comparison of runtimes
%timeit -r 1 -n 10 kmeanspar(gmdata1,20,6,6)
%timeit -r 1 -n 10 jit_kmeanspar(gmdata1,20,6,6)

10 loops, best of 1: 4.61 s per loop
10 loops, best of 1: 2.46 s per loop


**As the results above indicate, the use of jit can appreciably improve the computational speed of k-means||. However, jit's marginal improvement for k-means|| is worse that than observed from the use of broadcasting, so we will therefore use broadcasting as our preferred method.**

# Part V
# Perform K-means Algorithm and Compare Different Initialization Algorithms:

**Since using broadcasting to speed up makes the codes run fastest, we use the broadcasting version of the initialization algorithms to perform the kmeans algorithm.**

### 1. Real World Data--spambase:

**Perform on spambase:**

In [332]:
%%time
def vrandomkmeans(data,k):
    initial=kmeans(data,k)
    return vLloyd(data,initial)
rcentersspam,rlabelsspam,rispam=vrandomkmeans(spam,20)

CPU times: user 6.13 s, sys: 1.95 s, total: 8.08 s
Wall time: 8.1 s


In [333]:
%%time
def vppkmeans(data,k):
    initial=vkmeanspp(data,k)
    return vLloyd(data,initial)
ppcentersspam,pplabelspam,ppispam=vppkmeans(spam,20)

CPU times: user 1.5 s, sys: 908 ms, total: 2.41 s
Wall time: 2.44 s


In [334]:
%%time
def vscalablekmeans(data,k,l,r):
    initial=vkmeanspar(data,k,l,r)
    return vLloyd(data,initial)
parcentersspam,parlabelsspam,parispam=vscalablekmeans(spam,20,6,8)

CPU times: user 1.36 s, sys: 564 ms, total: 1.92 s
Wall time: 1.94 s


**The table of cost and iterations of different initialization algorithms on the spambase dataset:**

In [335]:
tablespam=pd.DataFrame({'cost':pd.Series([cost(spam,rcentersspam), cost(spam,ppcentersspam), 
                        cost(spam,parcentersspam)],index=['random', 'kmeans++', 'kmeans||']),
                        'iterations':pd.Series([rispam, ppispam, parispam], 
                        index=['random', 'kmeans++', 'kmeans||'])})
tablespam

,cost,iterations
random,1.529389e+08,226
kmeans++,2.318091e+07,43
kmeans||,2.425734e+07,32


### 2. Synthetic dataset--GaussMixture:

**2.1 gmdata1**

In [336]:
%%time
rcenters1,rlabels1,ri1=vrandomkmeans(gmdata1,20)

CPU times: user 684 ms, sys: 0 ns, total: 684 ms
Wall time: 693 ms


In [337]:
%%time
ppcenters1,pplabels1,ppi1=vppkmeans(gmdata1,20)

CPU times: user 1.54 s, sys: 0 ns, total: 1.54 s
Wall time: 1.55 s


In [338]:
%%time
parcenters1,parlabels1,pari1=vscalablekmeans(gmdata1,20,6,8)

CPU times: user 1.2 s, sys: 84 ms, total: 1.28 s
Wall time: 1.3 s


**The table of cost and iterations of different initialization algorithms on gmdata1:**

In [339]:
table1=pd.DataFrame({'cost':pd.Series([cost(gmdata1,rcenters1), cost(gmdata1,ppcenters1), 
                        cost(gmdata1,parcenters1)],index=['random', 'kmeans++', 'kmeans||']),
                        'iterations':pd.Series([ri1, ppi1, pari1], 
                        index=['random', 'kmeans++', 'kmeans||'])})
table1

,cost,iterations
random,145569.188581,32
kmeans++,145840.232731,52
kmeans||,147016.594974,37


**2.2 gmdata10**

In [340]:
%%time
rcenters10,rlabels10,ri10=vrandomkmeans(gmdata10,20)

CPU times: user 412 ms, sys: 4 ms, total: 416 ms
Wall time: 419 ms


In [341]:
%%time
ppcenters10,pplabels10,ppi10=vppkmeans(gmdata10,20)

CPU times: user 884 ms, sys: 0 ns, total: 884 ms
Wall time: 895 ms


In [342]:
%%time
parcenters10,parlabels10,pari10=vscalablekmeans(gmdata10,20,6,8)

CPU times: user 828 ms, sys: 44 ms, total: 872 ms
Wall time: 897 ms


**The table of cost and iterations of different initialization algorithms on gmdata10:**

In [343]:
table10=pd.DataFrame({'cost':pd.Series([cost(gmdata10,rcenters10), cost(gmdata10,ppcenters10), 
                        cost(gmdata10,parcenters10)],index=['random', 'kmeans++', 'kmeans||']),
                        'iterations':pd.Series([ri10, ppi10, pari10], 
                        index=['random', 'kmeans++', 'kmeans||'])})
table10

,cost,iterations
random,487544.522593,20
kmeans++,234821.209551,26
kmeans||,248286.442944,23


**2.3 gmdata100**

In [344]:
%%time
rcenters100,rlabels100,ri100=vrandomkmeans(gmdata100,20)

CPU times: user 500 ms, sys: 0 ns, total: 500 ms
Wall time: 502 ms


In [345]:
%%time
ppcenters100,pplabels100,ppi100=vppkmeans(gmdata100,20)

CPU times: user 440 ms, sys: 4 ms, total: 444 ms
Wall time: 444 ms


In [346]:
%%time
parcenters100,parlabels100,pari100=vscalablekmeans(gmdata100,20,6,8)

CPU times: user 992 ms, sys: 84 ms, total: 1.08 s
Wall time: 1.08 s


**The table of cost and iterations of different initialization algorithms on gmdata100:**

In [347]:
table100=pd.DataFrame({'cost':pd.Series([cost(gmdata100,rcenters100), cost(gmdata100,ppcenters100), 
                        cost(gmdata100,parcenters100)],index=['random', 'kmeans++', 'kmeans||']),
                        'iterations':pd.Series([ri100, ppi100, pari100], 
                        index=['random', 'kmeans++', 'kmeans||'])})
table100

,cost,iterations
random,1.934502e+06,22
kmeans++,1.497229e+05,2
kmeans||,4.111006e+05,26


# Part VI
# Discussion & Conclusion:

**For the algorithms above, using broadcasting is the best method to improve computational speed. Another, though less efficous method, is the use of jit as it can modestly improve the performance of k-means||. This result is understandable in the concext of this project as broadcasting is a useful means to improve the speed of matrix operations, many of which are employed by thele algorithms.**

In [348]:
#results table for spambase
tablespam

,cost,iterations
random,1.529389e+08,226
kmeans++,2.318091e+07,43
kmeans||,2.425734e+07,32


**The table above shows that, for the real world dataset spambase, the total costs resulting k-means++ and k-means|| are much smaller than the cost from using randomly initialized k-means. Also, the iterations times using k-means++ and k-means|| are also much smaller than using random selecting initialization algorithm.**

**In addition, the running time using k-means++ and k-means|| are much smaller than using randomly initialized k-means too.** 

**Therefore, k-means|| and k-means++ are better than randomly initialized k-means in producing initial set of centers for the dataset spambase.** 

In [349]:
#results table for gmdata1
table1

,cost,iterations
random,145569.188581,32
kmeans++,145840.232731,52
kmeans||,147016.594974,37


In [350]:
#results table for gmdata10
table10

,cost,iterations
random,487544.522593,20
kmeans++,234821.209551,26
kmeans||,248286.442944,23


In [351]:
#results table for gmdata100
table100

,cost,iterations
random,1.934502e+06,22
kmeans++,1.497229e+05,2
kmeans||,4.111006e+05,26


**The above three tables are for three synthetic dataset: gmdata1, gmdata10, gmdata100. These three data are very similar, only that the sphere variance of the simulated centers are different. The sphere variance scale of the simulated centers of gmdata1, gmdata10, gmdata100 are 1:10:100. Therefore, the gmdata100 is more spreaded widely than gmdata10 and gmdata10 is more spreaded widely than gmdata1.**

**The costs and iterations times for gmdata1 are very close for the three initialization algorithms. For gmdata10, the kmeans++ and kmeans|| are better than random selecting initialization algorithm in cost, however the cost rate between random selecting and other two initialization algorithms is not large(about 1.5). For gmdata100, the results are much more similar to the results of spambase. The cost ratio between random selecting and other two initialization algorithms is munch larger(about 4-5) and the iterations times for random selecting is much larger than the other two initialization algorithms.**

**Based on the above four tables, k-means++ and k-means|| are generally better initialization algorithms than random selection. As the points in the dataset spread more widely, the advantage of k-means++ and k-means|| also grows larger compared with random selection initialization. This makes intuitive sense as when the points in the dataset are spread relatively tightlf, the distribution of initial set of centers selected by k-means++ or k-means|| will not be that dissimilar from the initial set of centers chosen by random selection. However, as the points in the dataset are spread more widely, the distibution of initial set of centers produced by k-means++ or k-means|| will be more widely dispersed than initial set of centers produced by random selection, serving to increase the differnece in the results.**

**In general, the results for k-means++ and k-means|| are not dramatically different. The costs and iteration times using these two initialization algorithms are very close for these four datasets. The running time are also similar for k-means++ and k-means||. It is reasonable, because as the paper says, k-means|| is a parallel version of k-means++. As discussed at the begining, when the dataset is massive, k-means++ may be not appropriate and kmeans-|| will be a better initialization algorithm.**

# Part VII
# Reference:

**Bahmani B., B. Moseley, A. Vattani, R. Kuwar, and S. Vasslivitskii. (2012). Scaleable K-Means++.  Proceedings of the VLDB Endowment (PVLDB), Vol. 5, No. 7, pp. 622-633.**

**link to the repository: https://github.com/eastman74/STA-663-FINAL-PROJECT.git**